<a href="https://colab.research.google.com/github/99kenny/Deep-Learning/blob/main/VGGNet_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
# from google.colab import drive
# drive.mount('/content/drive')

In [74]:
!cd /content/drive/MyDrive/"Colab Notebooks"/DeepLearningImplementation/Deep-Learning

!git config --global user.email 'helloiamkenny@gmail.com'
!git config --global user.name '99kenny'

/bin/bash: line 0: cd: /content/drive/MyDrive/Colab Notebooks/DeepLearningImplementation/Deep-Learning: No such file or directory


# VGGNet
- dataset : CIFAR-10

In [75]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from keras.models import Model
from keras import optimizers, regularizers, utils
from keras.preprocessing.image import ImageDataGenerator
from keras import utils
from tensorflow.keras.utils import to_categorical

In [76]:
# download data
cifar10 = tf.keras.datasets.cifar10 
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog','frog', 'horse', 'ship', 'truck']

In [77]:
# fig = plt.figure(figsize=(20,20))
# axes = []
# for index, img in enumerate(train_images):
#   ax = fig.add_subplot(5,5,index+1)
#   ax.set_title(class_names[train_labels[index][0]])
#   ax.imshow(img)
#   axes.append(ax)
#   if (index+1) % 25 == 0:
#     break
# plt.show()

In [78]:
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

# split data -> validation, test
valid_images, valid_labels = test_images[0:5000,:], test_labels[0:5000,:]
test_images, test_labels = test_images[5000:,:], test_labels[5000:,:]

print(valid_images.shape)
print(valid_labels.shape)

(5000, 32, 32, 3)
(5000, 10)


In [79]:
# datagen = ImageDataGenerator(
#             featurewise_center=False,  # set input mean to 0 over the dataset
#             samplewise_center=False,  # set each sample mean to 0
#             featurewise_std_normalization=False,  # divide inputs by std of the dataset
#             samplewise_std_normalization=False,  # divide each input by its std
#             zca_whitening=False,  # apply ZCA whitening
#             rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
#             width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#             height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#             horizontal_flip=True,  # randomly flip images
#             vertical_flip=False,
#             )  # randomly flip images

# datagen.fit(train_images)

In [80]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False,
            )  # randomly flip images

datagen.fit(train_images)

In [81]:

weight_decay = 0.0005
img_rows, img_cols = 32, 32
img_channels = 3 

img_dim = (img_rows, img_cols, img_channels)  
img_input = Input(shape=img_dim)

def vgg_model():
  x = Conv2D(64,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(img_input) 
  x = Dropout(0.3)(x)
  x = Conv2D(64,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x) 
  x = BatchNormalization()(x)
  x = MaxPooling2D((2,2),strides=(2,2))(x) 


  x = Conv2D(128,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x)
  x = Dropout(0.4)(x)
  x = Conv2D(128,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x) 
  x = BatchNormalization()(x)
  x = MaxPooling2D((2,2),strides=(2,2))(x) 

  x = Conv2D(256,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x) 
  x = Dropout(0.4)(x)
  x = Conv2D(256,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x) 
  x = Conv2D(256,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x) 
  x = Conv2D(256,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x)
  x = BatchNormalization()(x) 
  x = MaxPooling2D((2,2),strides=(2,2))(x) 

  x = Conv2D(512,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x)
  x = Dropout(0.4)(x)
  x = Conv2D(512,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x) 
  x = Conv2D(512,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x)
  x = Conv2D(512,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x)
  x = BatchNormalization()(x) 
  x = MaxPooling2D((2,2),strides=(2,2))(x) 

  x = Conv2D(512,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x) 
  x = Dropout(0.4)(x)
  x = Conv2D(512,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x)
  x = Conv2D(512,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x)
  x = Conv2D(512,(3,3),padding='same',activation = 'relu',kernel_regularizer=regularizers.l2(weight_decay))(x)
  x = BatchNormalization()(x) 
  x = MaxPooling2D((2,2),strides=(2,2))(x)
  x = Dropout(0.5)(x)

  x = Flatten()(x)
  x = Dense(512, activation = 'relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(521, activation = 'relu')(x)
  x = Dropout(0.5)(x)
  x = Dense(10, activation = 'softmax')(x)
  return x


model = Model(inputs=img_input, outputs = vgg_model())
models = [model]

# Train

In [82]:
hist = []
for model in models:
  model.compile(optimizer='Adam', 
                loss='categorical_crossentropy', metrics=['accuracy'])


  from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
  learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                              patience=3, 
                                              verbose=1, 
                                              factor=0.1, 
                                              min_lr=0.00001)

  hist.append(model.fit(datagen.flow(train_images, train_labels, batch_size=128), epochs=100,
            validation_data=(valid_images, valid_labels),callbacks = [learning_rate_reduction]))

Epoch 1/100
391/391 [==============================] - 36s 88ms/step - loss: 4.2047 - accuracy: 0.1803 - val_loss: 4.3594 - val_accuracy: 0.1014 - lr: 0.0010
Epoch 2/100
391/391 [==============================] - 33s 85ms/step - loss: 2.8338 - accuracy: 0.2066 - val_loss: 2.8772 - val_accuracy: 0.1612 - lr: 0.0010
Epoch 3/100
391/391 [==============================] - 34s 86ms/step - loss: 2.3206 - accuracy: 0.2287 - val_loss: 2.2188 - val_accuracy: 0.2314 - lr: 0.0010
Epoch 4/100
391/391 [==============================] - 33s 85ms/step - loss: 2.0800 - accuracy: 0.2520 - val_loss: 2.0159 - val_accuracy: 0.2538 - lr: 0.0010
Epoch 5/100
391/391 [==============================] - 33s 85ms/step - loss: 1.9501 - accuracy: 0.2871 - val_loss: 2.5358 - val_accuracy: 0.1952 - lr: 0.0010
Epoch 6/100
391/391 [==============================] - 33s 85ms/step - loss: 1.8786 - accuracy: 0.3166 - val_loss: 1.9029 - val_accuracy: 0.2892 - lr: 0.0010
Epoch 7/100
391/391 [==============================]

# Test

In [83]:
result = []
for index,model in enumerate(models):
  test_loss, test_acc, test_sparse_top_k_categorical_accuracy = model.evaluate(test_images,test_labels)
  result.append([test_acc,test_sparse_top_k_categorical_accuracy])   
  print("model_", chr(index+65))
  print('test loss:', test_loss)
  print('top-1 test error : ', (1 - test_acc)*100)
  print('top-5 test error : ',(1 - test_sparse_top_k_categorical_accuracy)*100)

157/157 [==============================] - 2s 13ms/step - loss: 0.6241 - accuracy: 0.8430


ValueError: ignored

# Ensemble

# Git push

In [ ]:

%cd /content/drive/MyDrive/"Colab Notebooks"/DeepLearningImplementation/Deep-Learning
!ls
!git add .
!git commit -m "feat: vgg A~E"
!git push origin main
